# Almost Matching Exactly Data Cleaning

## Algorithm Selection
Select which algorithm you want to clean your dataset for. Note that DAME and FLAME have the same data requirements, as do MALTS and AHB.

In [1]:
import ipywidgets as widgets
algorithm = widgets.ToggleButtons(
    options=[('DAME / FLAME', 0), ('MALTS / AHB', 1)],
    description='algorithm:',
    disabled=False,
    tooltips=[
        'This option will result in a cleaned dataset compatible with DAME or FLAME', 
        'This option will result in a cleaned dataset compatible with MALTS or AHB'
    ]
)
display(algorithm)

ToggleButtons(description='algorithm:', options=(('DAME / FLAME', 0), ('MALTS / AHB', 1)), tooltips=('This opt…

## Import File
This field only accepts file paths ending in .csv or .xpt

In [2]:
filepath = widgets.Text(
    value='D:\Downloads\DEMO.XPT',
    placeholder='D:\Downloads\DEMO.XPT',
    description='file path:',
    disabled=False
)
display(filepath)

Text(value='D:\\Downloads\\DEMO.XPT', description='file path:', placeholder='D:\\Downloads\\DEMO.XPT')

## Select Columns of Interest
In this field, you can select as many columns as you want. Be sure to include the treatment and outcome columns in your selection, as well as all covariates of interest, because all other columns will be removed from the cleaned dataset.

In [3]:
import pandas as pd
# Read XPT or CSV into dataframe
if filepath.value.lower().endswith('xpt'):
    df = pd.read_sas(filepath.value)
elif filepath.value.lower().endswith('csv'):
    df = pd.read_csv(filepth.value)
else:
    raise Exception('Unsupported file type')
# Select columns of interest
columns = list(df.columns)
covariate_select = widgets.HBox([
    widgets.Label(value='Select all columns of interest:'), 
    widgets.SelectMultiple(
        options=columns,
        value=columns,
        disabled=False)
])
display(covariate_select)

## Input General Information
In this step, you will select which column contains information about whether a unit is treated or control and which column contains the outcome variable. You will also have the option to provide a set of comma-delimited missing indicators, which are values in the provided dataset which indicate missing data.

In [4]:
from ipywidgets import Layout
# Define layout
label = Layout(width='14rem', height='auto')
box = Layout(width='25rem', height='auto', margin='0px 2rem 0px 0px')
desc = Layout(width='50rem', height='auto')
# Retain columns of interest
df = df[list(covariate_select.children[1].value)]
columns = list(df.columns)
# Select treatment indicator
treatment_select = widgets.HBox([
    widgets.Label(value='treatment indicator:', layout=label), 
    widgets.Select(
        options=columns,
        value=columns[0],
        disabled=False,
        layout=box),
    widgets.VBox([
        widgets.HBox([
            widgets.Label(value='treatment condition:', layout=label),
            widgets.Text(
                value='x==1',
                placeholder='x==1',
                disabled=False,
                layout=box)
        ]),
        widgets.HBox([
            widgets.HTML(value="""NOTE: The treatment condition must be a boolean 
            function of x where x is a unit's value in the column selected as the 
            treatment indicator. The function should return True if the unit is treated
            and False otherwise. (e.g. x==1 or x>3)""", layout=desc),
        ])
    ])
    
])
display(treatment_select)
# Select outcome column
outcome_select = widgets.HBox([
    widgets.Label(value='outcome variable:', layout=label), 
    widgets.Select(
        options=columns,
        value=columns[0],
        disabled=False,
        layout=box)
])
display(outcome_select)
# Enter missing indicator
missing_indicator_input = widgets.HBox([
    widgets.Label(value='missing indicators:', layout=label),
    widgets.Text(
        value='',
        placeholder='Leave blank if not applicable',
        disabled=False,
        layout=box)
])
display(missing_indicator_input)
# Indicate any known continuous covariates
continuous_select = widgets.HBox([
    widgets.Label(value='continuous covariates:', layout=label), 
    widgets.SelectMultiple(
        options=columns,
        value=[],
        disabled=False,
        layout=box),
    widgets.HTML(value="""OPTIONAL: Select any covariates which you know are continuous. There is no need
    to select the outcome column, since that is already assumed to be continuous. Also note that the next
    code segment will identify other potentially continuous covariates.""", layout=desc)
])
if algorithm.value == 0:
    display(continuous_select)

## Data Cleaning
This code will clean the dataset according to your algorithm selection and general information inputs. If DAME/FLAME is selected, you may be prompted to handle potentially continuous covariates. The inputs will accept the following options:

How do you want to proceed?
- **ignore**: Leave the data for this column as it is in the original dataset
- **ignore all**: Leave the data for this column and all continuous columns following this one as they are in the original dataset
- **delete**: Delete this column from the cleaned dataset
- **delete all**: Delete this column and all continous columns following this one
- **bin**: Bin the data for this column
- **bin all**: Bin the data for this column and all continuous columns following this one *using the same binning options*

If you select **bin** or **bin all**, you will see the following prompt, which accepts these options:

Number of bins, set of boundaries, or binning method:
- If an integer *n* is provided, the data will be split into *n* equal-width bins
- If a set of comma-delimited values are provided, those values will be used as boundaries for binning, inclduing the rightmost edge
- If a string from the list below is provided, the bins will be computed with the selected method:
    - **auto**: Maximum of the ‘sturges’ and ‘fd’ estimators. Provides good all around performance.

    - **fd**: Freedman Diaconis Estimator - Robust (resilient to outliers) estimator that takes into account data variability and data size.

    - **doane**: An improved version of Sturges’ estimator that works better with non-normal datasets.

    - **scott**: Less robust estimator that that takes into account data variability and data size.

    - **stone**: Estimator based on leave-one-out cross-validation estimate of the integrated squared error. Can be regarded as a generalization of Scott’s rule.

    - **rice**: Estimator does not take variability into account, only data size. Commonly overestimates number of bins required.

    - **sturges**: R’s default method, only accounts for data size. Only optimal for gaussian data and underestimates number of bins for large non-gaussian datasets.

    - **sqrt**: Square root (of data size) estimator, used by Excel and other programs for its speed and simplicity.

In [ ]:
import numpy as np
# Extract treatment column name and treatment condition
treatment_column_name = treatment_select.children[1].value
treatment_condition = treatment_select.children[2].children[0].children[1].value
# Extract outcome column name
outcome_column_name = outcome_select.children[1].value
# Extract missing indicator
missing_indicator = missing_indicator_input.children[1].value
# Remove units with missingness in treatment or outcome columns
if missing_indicator:
    df.drop(df[df[treatment_column_name] == missing_indicator].index, inplace=True)
    df.drop(df[df[outcome_column_name] == missing_indicator].index, inplace=True)
# Apply treatment condition to treatment column
df[treatment_column_name] = df[treatment_column_name].apply(lambda x: 1 if eval(treatment_condition) else 0)
# Deal with any continuous data if algorithm selection requires categorical covariates
if algorithm.value == 0:
    option = ''
    bins = ''
    for col in [x for x in columns if x != treatment_column_name and x != outcome_column_name]:
        # Determine if a covariate is likely to be continuous
        if len(pd.unique(df[col])) > len(df[col]) / 10:
            if option[-3:] != 'all':
                print('WARNING: ' + str(col) + ' may be continuous')
                option = input("How do you want to proceed? ")
                if option not in ['ignore', 'ignore all', 'delete', 'delete all', 'bin', 'bin all']:
                    print('ERROR: Invalid input')
                    option = input("How do you want to proceed? ")
            if option == 'delete' or option == 'delete all':
                df = df.drop(columns=col)
                print(str(col) + ' was deleted')
            if option == 'bin' or (option == 'bin all' and bins == ''):
                bins = input('Number of bins, set of boundaries, or binning method: ')
                bins = list(bins.split(', '))
                if bins[0].isnumeric():
                    bins = list(map(int, bins))
                if len(bins) == 1:
                    bins = bins[0]
            if option == 'bin' or option == 'bin all':
                edges = np.histogram_bin_edges(df[col], bins, range=(df[col].min(), df[col].max()))
                df[col] = pd.cut(list(df[col]), edges, labels=False)
                print(str(col) + ' was binned')
            if option == 'ignore' or option == 'ignore all':
                print(str(col) + ' was ignored')
    print('SUCCESS: Data cleaned for DAME/FLAME')
else :
    print('SUCCESS: Data cleaned for MALTS/AHB')

## Download Cleaned Dataset
Provide a destination file path for the cleaned dataset that ends in .csv and download! You are now ready to use your selected algorithm.

In [ ]:
destination = widgets.HBox([
    widgets.Label(value='destination file path:'),
    widgets.Text(
        value=filepath.value[:-4] + '_CLEANED.csv',
        placeholder='D:\Downloads\DEMO_CLEANED.csv',
        disabled=False)
])
display(destination)
button = widgets.Button(
    description='Download',
    disabled=False,
    tooltip='Download',
    icon='download'
)
display(button)

def download(button):
    # Extract destination file path
    destination_path = destination.children[1].value
    # Download cleaned data
    df.to_csv(destination_path, index=False)
    print('Cleaned data downloaded to ' + destination_path)
button.on_click(download)